In [1]:
import sys
import json
from fabric import Connection
from invoke import Responder, UnexpectedExit

sys.version

'3.8.10 (default, May 19 2021, 18:05:58) \n[GCC 7.3.0]'

In [2]:
result = Connection('ubuntu@128.1.135.208').run('hostname', hide=False)

swarm-01


In [3]:
result.stdout

'swarm-01\n'

In [4]:
hosts = [
    "118.193.45.223",
    "128.1.135.208",
    "103.14.34.147",
    "23.91.97.32",
    "152.32.210.117",
    "23.91.98.101",
    "152.32.189.174",
    "36.255.220.243",
    "152.32.192.130",
    "101.36.117.63",
    "101.36.120.12",
    "101.36.122.83",
    "152.32.215.147",
    "152.32.251.236",
    "152.32.190.173",
    "23.91.101.163",
    "118.193.39.223",
    "165.154.3.36",
    "45.249.247.180",
    "152.32.210.173",
    "23.91.100.167",
    "45.249.247.46",
    "128.1.138.92",
]

In [5]:
hosts

['118.193.45.223',
 '128.1.135.208',
 '103.14.34.147',
 '23.91.97.32',
 '152.32.210.117',
 '23.91.98.101',
 '152.32.189.174',
 '36.255.220.243',
 '152.32.192.130',
 '101.36.117.63',
 '101.36.120.12',
 '101.36.122.83',
 '152.32.215.147',
 '152.32.251.236',
 '152.32.190.173',
 '23.91.101.163',
 '118.193.39.223',
 '165.154.3.36',
 '45.249.247.180',
 '152.32.210.173',
 '23.91.100.167',
 '45.249.247.46',
 '128.1.138.92']

In [6]:
# Check status

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run("curl -s http://localhost:1635/peers | jq '.peers | length'", hide=True)
    print(f'peers = {result.stdout.strip()}')

==== 00/23 : 118.193.45.223 ====
swarm-00
peers = 0
==== 01/23 : 128.1.135.208 ====
swarm-01
peers = 3
==== 02/23 : 103.14.34.147 ====
swarm-02
peers = 0
==== 03/23 : 23.91.97.32 ====
swarm-03
peers = 1
==== 04/23 : 152.32.210.117 ====
swarm-04
peers = 0
==== 05/23 : 23.91.98.101 ====
swarm-05
peers = 26
==== 06/23 : 152.32.189.174 ====
swarm-06
peers = 0
==== 07/23 : 36.255.220.243 ====
swarm-07
peers = 0
==== 08/23 : 152.32.192.130 ====
swarm-08
peers = 2
==== 09/23 : 101.36.117.63 ====
swarm-09
peers = 
==== 10/23 : 101.36.120.12 ====
swarm-10
peers = 0
==== 11/23 : 101.36.122.83 ====
swarm-11
peers = 15
==== 12/23 : 152.32.215.147 ====
swarm-12
peers = 0
==== 13/23 : 152.32.251.236 ====
swarm-13
peers = 0
==== 14/23 : 152.32.190.173 ====
UHost14
peers = 23
==== 15/23 : 23.91.101.163 ====
UHost15
peers = 32
==== 16/23 : 118.193.39.223 ====
UHost16
peers = 25
==== 17/23 : 165.154.3.36 ====
UHost17
peers = 16
==== 18/23 : 45.249.247.180 ====
UHost18
peers = 0
==== 19/23 : 152.32.210.1

In [ ]:
# Check peer balances

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run('cd swarm-single-machine && ./balances.sh')
    print()

In [7]:
# All down

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && ./down.sh')

==== 00/23 : 118.193.45.223 ====
swarm-00


Stopping swarm-single-machine_bee-1_1  ... 
Stopping swarm-single-machine_clef-1_1 ... 
Stopping swarm-single-machine_bee-1_1  ... done
Stopping swarm-single-machine_clef-1_1 ... done
Removing swarm-single-machine_bee-1_1  ... 
Removing swarm-single-machine_clef-1_1 ... 
Removing swarm-single-machine_clef-1_1 ... done
Removing swarm-single-machine_bee-1_1  ... done
Removing network swarm-single-machine_default


==== 01/23 : 128.1.135.208 ====
swarm-01


Stopping swarm-single-machine_bee-1_1  ... 
Stopping swarm-single-machine_clef-1_1 ... 
ping swarm-single-machine_clef-1_1 ... doneRemoving swarm-single-machine_bee-1_1  ... 
Removing swarm-single-machine_clef-1_1 ... 
ving swarm-single-machine_bee-1_1  ... doneRemoving network swarm-single-machine_default


==== 02/23 : 103.14.34.147 ====
swarm-02


...

sudo: unable to resolve host UHost20: Name or service not known
Stopping swarm-single-machine_bee-1_1  ... 
Stopping swarm-single-machine_clef-1_1 ... 
ping swarm-single-machine_clef-1_1 ... doneRemoving swarm-single-machine_bee-1_1  ... 
Removing swarm-single-machine_clef-1_1 ... 
ving swarm-single-machine_bee-1_1  ... doneRemoving network swarm-single-machine_default


==== 21/23 : 45.249.247.46 ====
UHost21


sudo: unable to resolve host UHost21: Name or service not known
Stopping swarm-single-machine_bee-1_1  ... 
Stopping swarm-single-machine_clef-1_1 ... 
ping swarm-single-machine_clef-1_1 ... doneRemoving swarm-single-machine_bee-1_1  ... 
Removing swarm-single-machine_clef-1_1 ... 
Removing network swarm-single-machine_defaultone


==== 22/23 : 128.1.138.92 ====
UHost22


Stopping swarm-single-machine_bee-1_1  ... 
Stopping swarm-single-machine_clef-1_1 ... 
ping swarm-single-machine_clef-1_1 ... doneRemoving swarm-single-machine_bee-1_1  ... 
Removing swarm-single-machine_clef-1_1 ... 
Removing network swarm-single-machine_defaultone


In [ ]:
# All up

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && git pull')
    conn.run("cd swarm-single-machine && ./up.sh")

In [ ]:
# All restart
for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    with Connection(f'ubuntu@{host}') as conn:
        conn.run('hostname')
        conn.run('cd swarm-single-machine && ./restart.sh')

In [ ]:
# Rename hostname

sudopass = Responder(
    pattern=r'Password:',
    response='[please change]\n')

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run(f'hostnamectl set-hostname swarm-{i:02}', pty=True, watchers=[sudopass])
    conn.run('hostnamectl')
    print()

In [ ]:
# Add self hostname to /etc/hosts
# (to solve sudo error)

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('echo $(hostname -I | cut -d\  -f1) $(hostname) | sudo tee -a /etc/hosts')
    conn.run('cat /etc/hosts')

In [ ]:
# All addresses

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && ./addresses.sh')

In [ ]:
# Print all eth addresses

for _, host in enumerate(hosts):
    conn = Connection(f'ubuntu@{host}')
    result = conn.run('cd swarm-single-machine && ./addresses.sh', hide=True)
    payload = json.loads(result.stdout.strip())
    print(payload['ethereum'])

In [ ]:
# Try to init all

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    try:
        conn.run('git clone https://github.com/renning22/swarm-single-machine.git && cd swarm-single-machine && ./init.sh')
    except UnexpectedExit:
        print('Skip...')

In [ ]:
# Restart unhealthy

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run("curl -s http://localhost:1635/peers | jq '.peers | length'", hide=True)
    print(f'peers = {result.stdout.strip()}')
    if not result.stdout.strip():
        print('Restarting...')
        conn.run('cd swarm-single-machine && ./restart.sh')

In [ ]:
# Add swap-endpoint to hosts table (UCloud-HK-specific)
for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('printf "\n10.7.17.210 swap-endpoint-01\n" | sudo tee -a /etc/hosts')
    conn.run('cat /etc/hosts')

In [ ]:
import pandas as pd
import urllib3

def get(k, port=1635):
    http = urllib3.PoolManager()
    r = http.request('Get', f'http://localhost:{port}/{k}')
    return json.loads(r.data)

balances = pd.DataFrame(get('balances')['balances'])
balances['balance'].max()

In [ ]:
peer = '03b6a4ee224240c4777966fd4765c3b1dc079fd8086fa8850c05dae0d2dce6cf'
get(f'chequebook/cheque/{peer}')

In [ ]:
peer = '24a75108ed7659237b31c4f7066ccf2d2391087550b2e849517e04e63948ba9a'
get(f'chequebook/cheque/{peer}')

In [ ]:
peer = '24ac8ae73439aed048472dce577caa561ee54f2637d68a0313ff3441ec78c3b6'
get(f'chequebook/cheque/{peer}')

In [10]:
# Zip and backup all clef keys

sudopass = Responder(
    pattern=r'Password:',
    response='chainpion2021\n')

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run(f'sudo tar -czvf swarm-{i:02}.tar.gz /data/clef/', pty=True, watchers=[sudopass])
    print()

==== 00/23 : 118.193.45.223 ====
swarm-00
tar: Removing leading `/' from member names
/data/clef/
/data/clef/password
/data/clef/f6e2ff04b8e62f10fc13/
/data/clef/f6e2ff04b8e62f10fc13/credentials.json
/data/clef/f6e2ff04b8e62f10fc13/config.json
/data/clef/keystore/
/data/clef/keystore/UTC--2021-06-03T11-06-15.430703694Z--19517c4542133b455a73368122a7a61ffcb53d33
/data/clef/masterseed.json

==== 01/23 : 128.1.135.208 ====
swarm-01
tar: Removing leading `/' from member names
/data/clef/
/data/clef/keystore/
/data/clef/keystore/UTC--2021-06-04T08-52-11.053192811Z--84b134576e7841d3a77fddd629212273a3b2f45c
/data/clef/masterseed.json
/data/clef/password
/data/clef/5195a8bef7a7bd30a528/
/data/clef/5195a8bef7a7bd30a528/credentials.json
/data/clef/5195a8bef7a7bd30a528/config.json

==== 02/23 : 103.14.34.147 ====
swarm-02
tar: Removing leading `/' from member names
/data/clef/
/data/clef/keystore/
/data/clef/keystore/UTC--2021-06-04T10-00-28.541518957Z--0a579b5e7d221668d48642387be24d0845aed796
/da

UHost20
sudo: unable to resolve host UHost20: Name or service not known
tar: Removing leading `/' from member names
/data/clef/
/data/clef/password
/data/clef/masterseed.json
/data/clef/keystore/
/data/clef/keystore/UTC--2021-06-17T10-09-24.070416230Z--166cbdb8c6f701f4630f7a2467b0b7ab948e7cee
/data/clef/4396471082ab981e530d/
/data/clef/4396471082ab981e530d/credentials.json
/data/clef/4396471082ab981e530d/config.json

==== 21/23 : 45.249.247.46 ====
UHost21
sudo: unable to resolve host UHost21: Name or service not known
tar: Removing leading `/' from member names
/data/clef/
/data/clef/b28342ebdba48b1bdd25/
/data/clef/b28342ebdba48b1bdd25/credentials.json
/data/clef/b28342ebdba48b1bdd25/config.json
/data/clef/password
/data/clef/keystore/
/data/clef/keystore/UTC--2021-06-17T10-12-57.341746135Z--3ecdfac2778a48094b89ae6d9ec451762682d6ec
/data/clef/masterseed.json

==== 22/23 : 128.1.138.92 ====
UHost22
tar: Removing leading `/' from member names
/data/clef/
/data/clef/masterseed.json
/dat